# Wordle solver

Aswin van Woudenberg (https://www.aswinvanwoudenberg.com | https://github.com/afvanwoudenberg)

In [13]:
import json

import numpy as np
import pandas as pd
import ipywidgets as widgets

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from warnings import simplefilter
from collections import deque

In [14]:
with open('wordle.json', 'r') as fp:
    tree = json.load(fp)

In [15]:
traverse_tree = tree
active_row = 0

In [16]:
def on_btn_click(b, row, col):
    global traverse_tree
    global active_row
    
    for r in range(6):
        enable_row(r, r==row)
    
    if active_row < row:
        traverse_tree = traverse_tree[get_word(row - 1)][get_pattern(row - 1)]
        active_row = row
    
    if b.style.button_color == '#FFFFFF':
        b.style.button_color = '#C9B458'
    elif b.style.button_color == '#C9B458':
        b.style.button_color = '#787C7E'
    elif b.style.button_color == '#787C7E':
        b.style.button_color = '#6AAA64'
    elif b.style.button_color == '#6AAA64':
        b.style.button_color = '#C9B458'
    
    if row < 6:
        pattern = get_pattern(row)
        if pattern:
            set_word(row + 1, get_next_word(row).upper())
        else:
            set_word(row + 1, "     ")
    
        enable_row(row + 1, get_word(row + 1).strip())

In [17]:
def get_next_word(row):
    try:
        if isinstance(traverse_tree[get_word(row)][get_pattern(row)],str):
            return traverse_tree[get_word(row)][get_pattern(row)]
        return next(iter(traverse_tree[get_word(row)][get_pattern(row)]))
    except:
        return "     "
    return "     "

In [18]:
def create_on_btn_click_fun(row, col):
    def _on_btn_click(b):
        on_btn_click(b, row, col)
    return _on_btn_click

In [19]:
def set_word(row, word):
    for c in range(5):
        vbox.children[row].children[c].description = word[c]

In [20]:
def enable_row(row,b):
    for c in range(5):
        vbox.children[row].children[c].disabled = not b

In [21]:
def get_pattern(row):
    pattern = []
    for c in range(5):
        if vbox.children[row].children[c].style.button_color == '#FFFFFF':
            return None
        elif vbox.children[row].children[c].style.button_color == '#C9B458':
            pattern.append('P')
        elif vbox.children[row].children[c].style.button_color == '#787C7E':
            pattern.append('F')
        elif vbox.children[row].children[c].style.button_color == '#6AAA64':
            pattern.append('T')
    return ''.join(pattern)

In [22]:
def get_word(row):
    letters = []
    for c in range(5):
        letters.append(vbox.children[row].children[c].description)
    return ''.join(letters).lower()

In [23]:
def on_reset_button_click(b):
    global traverse_tree
    global active_row
    for r in range(6):
        enable_row(r, r==0)
        for c in range(5):
            b = vbox.children[r].children[c]
            b.style.button_color = '#FFFFFF'
            b.description = ' '
    traverse_tree = tree
    active_row = 0
    set_word(0, next(iter(traverse_tree)).upper())

In [24]:
vbox_items = []
for r in range(6):
    hbox_items = []
    for c in range(5):
        button = widgets.Button(description=' ', style=dict(button_color='#FFFFFF', font_weight='bold'), layout=widgets.Layout(width='32px', height='32px', border='solid 1px'))
        button.on_click(create_on_btn_click_fun(r,c))
        hbox_items.append(button)
    vbox_items.append(widgets.HBox(hbox_items))

vbox = widgets.VBox(vbox_items)
reset_button = widgets.Button(description="Reset")
reset_button.on_click(on_reset_button_click)
display(vbox, reset_button)
on_reset_button_click(reset_button)

Button(description='Reset', style=ButtonStyle())

Enter the suggested word into Wordle. Click the letters to change the colors and get the next guess. 